In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import extinction
from astroquery.ipac.irsa.irsa_dust import IrsaDust
import astropy.coordinates as coord
import astropy.units as u


def preprocess(filename, bin=False, display=False):
    # lightcurve data
    curve = pd.read_csv("./TESS_data/light_curves_fausnaugh/" + filename, delim_whitespace=True)
    curve_name = filename.split("_")[1]
    # information about transient
    curve_meta = transients[transients['IAU'] == curve_name]
    if curve_meta.empty:
        return None, None

    # convert time to relative to discovery
    curve['relative_time'] = curve['TJD'] - curve_meta["TJD_discovery"].iloc[0]
    curve.index = pd.TimedeltaIndex(curve['relative_time'], unit="D")

    # sigma clipping
    for _ in range(0, 5):
        uncert_mean = curve.e_cts.mean()
        threshold = 3*curve.e_cts.std()
        curve = curve[np.abs(curve['e_cts'] - uncert_mean) <= threshold]

    # sub bg flux
    if not curve['bkg_model'].isnull().all():
        curve['cts'] = curve['cts'] - curve['bkg_model']

    # correct milky way extinction
    # Set relevant parameters
    ra = curve_meta["ra"].iloc[0]
    dec = curve_meta["dec"].iloc[0]
    flux_in = curve["cts"]
    fluxerr_in = curve["e_cts"]
    bandpass_wavelengths = np.array([786.5,])

    # Get Milky Way E(B-V) Extinction
    coo = coord.SkyCoord(ra * u.deg, dec * u.deg, frame='icrs')
    dust = IrsaDust.get_query_table(coo, section='ebv')
    mwebv = dust['ext SandF mean'][0]

    # Remove extinction from light curves
    # (Using negative a_v so that extinction.apply works in reverse and removes the extinction)
    extinction_per_passband = extinction.fitzpatrick99(wave=bandpass_wavelengths, a_v=-3.1 * mwebv, r_v=3.1, unit='aa')
    flux_out = extinction.apply(extinction_per_passband[0], flux_in, inplace=False)
    fluxerr_out = extinction.apply(extinction_per_passband[0], fluxerr_in, inplace=False)

    curve['cts'] = flux_out
    curve['e_cts'] = fluxerr_out

    # normalize
    def normalize(cts, e_cts):
        curve['cts'] = (cts- cts.min()) / (cts.max()-cts.min())
        curve['e_cts'] = e_cts / (cts.max()-cts.min())

    normalize(curve['cts'], curve['e_cts'])

    # bin
    if bin:
        curve = curve.resample("1D").mean()


    id_max = curve["cts"].idxmax()

    max_data = curve.loc[id_max, :]
    t_max = max_data['relative_time']

    if display:
        plot_title = f"{curve_name}\n Class: {curve_meta['class'].iloc[0]}, Sector: {curve_meta['sector'].iloc[0]} \nCoords:{curve_meta['ra'].iloc[0], curve_meta['dec'].iloc[0]}, \nDiscovery TJD: {curve_meta['TJD_discovery'].iloc[0]}, Survey: {curve_meta['survey'].iloc[0]}"
        ax = curve.plot.scatter(x="relative_time", y='cts', c="00000", alpha=0.5, yerr='e_cts', ylabel="Flux", xlabel="Days relative to discovery", title=plot_title)
        ax.axvline(t_max, color="red", linestyle="--")

    return curve, t_max


transients = pd.read_csv("./TESS_data/AT_count_transients_s1-47 (4).txt", names=["sector", "ra", "dec", "mag", "TJD_discovery", "type" ,"class", "IAU", "survey", "cam", "ccd", "col", "row"], delim_whitespace=True)
light_curves = os.listdir("./TESS_data/light_curves_fausnaugh")

no_info = []
import time

start = time.perf_counter()
for file in light_curves:
    print(file)
    light_curve, max_light = preprocess(file)

    # light curves with no data about them AT_count
    if light_curve is None:
        no_info.append(file)
print(no_info)
input_image = None #TODO


lc_2018eel_cleaned
lc_2018emt_cleaned
lc_2018eny_cleaned
lc_2018eoa_cleaned
lc_2018eoc_cleaned
lc_2018eod_cleaned
lc_2018eph_cleaned
lc_2018eqo_cleaned
lc_2018esp_cleaned
lc_2018esw_cleaned
lc_2018evo_cleaned
lc_2018exc_cleaned
lc_2018fbm_cleaned
lc_2018fdw_cleaned
lc_2018fdx_cleaned
lc_2018fgb_cleaned
lc_2018fhs_cleaned
lc_2018fhw_cleaned
lc_2018fhw_detrended_cleaned
lc_2018flj_cleaned
lc_2018fll_cleaned
lc_2018flq_cleaned
lc_2018flr_cleaned
lc_2018flt_cleaned
lc_2018flz_cleaned
lc_2018fmg_cleaned
lc_2018fmm_cleaned
lc_2018fmn_cleaned
lc_2018fns_cleaned
lc_2018fqn_cleaned
lc_2018ftf_cleaned
lc_2018fub_cleaned
lc_2018fvi_cleaned
lc_2018fvi_detrended_cleaned
lc_2018fwd_cleaned
lc_2018fwi_cleaned
lc_2018fwk_cleaned
lc_2018fwm_cleaned
lc_2018fwr_cleaned
lc_2018fxj_cleaned
lc_2018fxl_cleaned
lc_2018fzi_cleaned
lc_2018gey_cleaned
lc_2018gfj_cleaned
lc_2018gfk_cleaned
lc_2018gku_cleaned
lc_2018glp_cleaned
lc_2018glq_cleaned
lc_2018glv_cleaned
lc_2018glw_cleaned
lc_2018goa_cleaned
lc_2018gqc_

KeyboardInterrupt: 